# Julia Review (ECON627 UBC)

****

If we choose to store our packages (and their versions), we want to activate our Project.toml file. 

In [1]:
using Pkg
Pkg.activate(joinpath(pwd(),"..")) ;

  Activating project at `c:\Users\steve\Documents\GitHub\ECON627_UBC.jl`


****

In this course you learned that there is a large class of estimators that, under suitable regularity conditions, admit an asymptotic linear representation: 

\begin{align*}
\sqrt n \left( \hat{\theta}_n - \theta_0 \right) &= - \left[ \frac{\partial Q_n(\theta_0)}{\partial \theta \partial \theta'} \right]^{-1} \sqrt n \frac{\partial Q_n(\theta_0)}{\partial \theta} + o_P(1)  \\
\sqrt n \left( \hat{\theta}_n - \theta_0 \right) &= \frac{1}{\sqrt n} \sum_{i=1}^n \xi_i + o_P(1) \quad , 
\end{align*}

where $\xi_i$ is known as the influence function, which are i.i.d and centered at zero $\mathbf{E}[\xi_i]=0$. By the CLT of i.i.d observations we have that 

\begin{equation}
\sqrt n \left( \hat{\theta}_n - \theta_0 \right) \rightarrow_d \mathcal{N}( 0, \mathbf{E}[\xi_i \xi_i'])
\end{equation}



**************
#### How to know if I'm missing an extra "n" when I compute the standard errors?
Just follow these steps:

- Compute analytically the asymptotic variance.
- Replace every expectation by sample average (every sample average must be divided by n!).
- The standard errors will be ``sqrt.(diag(avar))/sqrt(n)``


***
Before we do anything, we set a random seed to ensure replicability of our code.

In [2]:
using Random
Random.seed!(1234);

Load the ECON627 package 

In [3]:
using ECON627_UBC

## 1. Least Squares 

 - Population Criterion : $Q(\theta) = \mathbf{E} \left[ (Y_i - X_i'\theta)^2 \right]/2$
 - Sample Criterion: $Q_n(\theta) = \frac{1}{n} \sum_{i=1}^n (Y_i - X_i'\theta)^2/2$
 - Gradient:  $\frac{\partial Q_n(\theta_0)}{\partial \theta} = - \frac{1}{n} \sum_{i=1}^n (Y_i-X_i'\theta_0)X_i $
 - Hessian: $\frac{\partial Q_n(\theta_0)}{\partial \theta \partial \theta'} = \frac{1}{n} \sum_{i=1}^n X_i X_i' \rightarrow_p \mathbf{E} X_i X_i'$
 - Influence function: $\xi_i :=   (\mathbf{E}X_i X_i')^{-1} X_i U_i$ 
 
Therefore, the asymptotic variance is given by 
$ (\mathbf{E}X_i X_i')^{-1} \mathbf{E}[ U_i^2 X_i X_i'] (\mathbf{E}X_i X_i')^{-1}  $

In [4]:
using Parameters, LinearAlgebra, Distributions

In [5]:
function data_generating_ols(;n=1000)
    #Define the Multivariate Normal Distribution instance as an example
    mvnormal = MvNormal([1.0; 2.0], [1 0.7; 0.7 1])
    
    #Matrix X (n x 2 ) and ε (n x 1)
    X = rand(mvnormal,n)
    X = transpose(X)
    ε = randn(n, 1)
    
    #Vector y
    y = X*[1.0;1.0] + ε
        
    #Return the data
    return (X = X , y = y)
end    

data_generating_ols (generic function with 1 method)

In [6]:
@unpack X,y = data_generating_ols(n=1000)
@unpack θhat, se = ols(X,y)

(θhat = [0.9563864710176802; 1.0323477434590973;;], se = [0.0013591890782320168, 0.0008250537779697441])

In [11]:
n=1000
R=10^3

θcoverage95 = zeros(R)

Threads.@threads for r=1:R
    
    @unpack X,y = data_generating_ols(;n=n)
    @unpack θhat, se = ols(X,y)
    lower_ci  = θhat .- 1.96.*se
    upper_ci  = θhat .+ 1.96.*se
    
    #Coverage of θ[1]
    θcoverage95[r] = (lower_ci[1]<1.0)*(upper_ci[1]>1.0)
end

sum(θcoverage95)/R

0.048

## 2. Linear GMM

 - Population Criterion : $Q(\theta) = \mathbf{E} \left[ Z_i (Y_i - X_i'\theta) \right]' A'A \mathbf{E} \left[ Z_i (Y_i - X_i'\theta) \right]/2$ 
 
 Notice that if there's correct specification, $\mathbf{E}  \left[ Z_i (Y_i - X_i'\theta) \right] = 0$ ,this population criterion has a minimum value of 0 for any choice of norm $A$.
 
 - Sample Criterion: $Q_n(\theta) = \frac{1}{n}  \left[ \sum_{i=1}^n Z_i (Y_i - X_i'\theta) \right]' A_n'A_n  \left[ \sum_{i=1}^n Z_i (Y_i - X_i'\theta) \right]/2$
 - Gradient:  $\frac{\partial Q_n(\theta_0)}{\partial \theta'} = - \frac{1}{n} \sum_{i=1}^n X_i Z_i' A_n' A_n \frac{1}{n} \sum_{i=1}^n Z_i(Y_i-X_i'\theta_0)$
 - Hessian: $\frac{\partial Q_n(\theta_0)}{\partial \theta \partial \theta'} = \frac{1}{n} \sum_{i=1}^n X_i Z_i' A_n'A_n  \frac{1}{n} \sum_{i=1}^n Z_i X_i' \rightarrow_p \Gamma_0 A'A \Gamma_0$
 - Influence function: $\xi_i :=   (\Gamma_0' A'A \Gamma_0)^{-1} \Gamma_0' A'A Z_i U_i$ 
 
Therefore, the asymptotic variance is given by 
$ (\Gamma_0' A'A \Gamma_0)^{-1} \Gamma_0' A'A \mathbf{E}[ U_i^2 Z_i Z_i'] A'A \Gamma_0 (\Gamma_0' A'A \Gamma_0)^{-1}  $

In [8]:
using Parameters, LinearAlgebra, Distributions

In [9]:
function data_generating_gmm(;n=1000)
   
    ϵ=randn(n,1)
    V=randn(n,1)
    U=0.5*ϵ+V
    
    Z1=randn(n,1)
    Xexo=randn(n,3)
    
    Xendo=Z1+Xexo*ones(3,1)+V
    Y=Xendo+Xexo*ones(3,1)+U

    return (Y=Y, Xendo = Xendo, Xexo=Xexo, Z1 = Z1)  
end    

data_generating_gmm (generic function with 1 method)

In [10]:
function estimate_twosls(Y,X,Z)
    #Extract sample size
    n = size(Y, 1)
    
    #Step 1: Estimate theta
    Q=Z'*X
    W=inv(Z'*Z)
    θhat= inv(Q'*W*Q)*Q'*W*(Z'*Y)
    
    res = Y - X * θhat

    #Step 2: Compute Avar (Heteroskedastic Robust) 
    
    zr = Z .* res  
    avar = inv(Q'*W*Q/n)* Q'*W*(zr'*zr/n)*W*Q*inv(Q'*W*Q/n) 
    se = sqrt.(diag(avar))

    return (θhat=θhat , se = se )
end

estimate_twosls (generic function with 1 method)

In [11]:
@unpack Xendo, Xexo,Y,Z1 = data_generating_gmm(n=1000)
X=hcat(Xendo,Xexo)
Z=hcat(Xexo,Z1)
@unpack θhat, se = estimate_twosls(Y,X,Z)

(θhat = [0.9938392308323604; 0.990901289906396; 0.9785738392953602; 0.9930816515690295;;], se = [1.1176338684691591, 1.5939775190723664, 1.5579986598162396, 1.5706964438137758])

In [12]:
n=1000
R=10^3

θcoverage95 = zeros(R)

Threads.@threads for r=1:R
    
    @unpack Xendo, Xexo,Y,Z1 = data_generating_gmm(;n=n)
    X=hcat(Xendo,Xexo)
    Z=hcat(Xexo,Z1)
    @unpack θhat, se = estimate_twosls(Y,X,Z)
    
    lower_ci  = θhat .- 1.96.*se/sqrt(n)
    upper_ci  = θhat .+ 1.96.*se/sqrt(n)
    θcoverage95[r] = (lower_ci[1]<1.0)*(upper_ci[1]>1.0)
end

sum(θcoverage95)/R

0.943

## 3. Non-linear GMM

 - Population Criterion : $Q(\theta) = \mathbf{E} \left[g(W_i,\theta) \right]' A'A \mathbf{E} \left[ g(W_i,\theta) \right]/2$ 
 
 Notice that if there's correct specification, $\mathbf{E}  \left[ g(W_i,\theta)\right] = 0$ ,this population criterion has a minimum value of 0 for any choice of norm $A$.
 
 - Sample Criterion: $Q_n(\theta) = \frac{1}{n}  \left[ \sum_{i=1}^n g(W_i,\theta) \right]' A_n'A_n  \left[ \sum_{i=1}^n g(W_i,\theta) \right]/2$
 - Gradient:  $\frac{\partial Q_n(\theta_0)}{\partial \theta} =  \left( \frac{1}{n} \sum_{i=1}^n \frac{\partial g(W_i,\theta_0)'}{\partial \theta}\right) A_n'A_n' \left( \frac{1}{n} \sum_{i=1}^n  g(W_i,\theta_0)\right)$
 - Hessian: $\frac{\partial Q_n(\theta_0)}{\partial \theta \partial \theta'} = \left( \frac{1}{n} \sum_{i=1}^n \frac{\partial g(W_i,\theta_0)}{\partial \theta}\right) A_n'A_n' \left( \frac{1}{n} \sum_{i=1}^n \frac{\partial g(W_i,\theta_0)}{\partial \theta'}\right) + \left[I_k \otimes \left( \frac{1}{n} \sum_{i=1}^n  g(W_i,\theta_0)\right) A_n'A_n\right] \left[ \frac{1}{n} \sum_{i=1}^n \frac{\partial}{\partial \theta'} vec \left( \frac{\partial g(W_i,\theta_0)}{\partial \theta'} \right) \right]$
 $\rightarrow_p \Gamma_0' A'A\Gamma_0 + \left[I_k \otimes \mathbf{E}[g(W_i,\theta_0)'A'A] \right] \mathbf{E} \frac{\partial}{\partial \theta'}vec \left( \frac{\partial g(W_i,\theta_0)}{\partial \theta'} \right)  $
 
 where the second term is 0 if correctly specified. In the influence function below I'll assume it's zero to avoid writing too much algebra.
 
 - Influence function: $\xi_i :=   (\Gamma_0' A'A \Gamma_0)^{-1} \Gamma_0' A'A g(W_i,\theta_0)$ 
 
Therefore, the asymptotic variance is given by 
$ (\Gamma_0' A'A \Gamma_0)^{-1} \Gamma_0' A'A \mathbf{E}[ g(W_i,\theta_0) g(W_i,\theta_0)'] A'A \Gamma_0 (\Gamma_0' A'A \Gamma_0)^{-1}  $

In [13]:
using Parameters, LinearAlgebra, Distributions, Optim, ForwardDiff

In [14]:
function data_generating_gmm(;n=1000)
   
    ϵ=randn(n,1)
    V=randn(n,1)
    U=0.5*ϵ+V

    #2 instruments for 1 endogenous : overidentified
    Z1=randn(n,2)
    Xexo=randn(n,3)
    
    Xendo=Z1*ones(2,1)+Xexo*ones(3,1)+V
    Y=Xendo+Xexo*ones(3,1)+U

    return (Y=Y, Xendo = Xendo, Xexo=Xexo, Z1 = Z1)  
end    

data_generating_gmm (generic function with 1 method)

In [15]:
function estimate_nlgmm(Y, X, Z,W) 
    # Obtain sample size
    n = size(Y, 1)

    #Set up moment condition
    g = (y,x,z,θ) -> z*(y-x*θ)
    
    # Objective function based on moment condition
    function sample_moment(Y,X,Z,θ)
        v = map(i -> g(Y[i],X[i, :]',Z[i,:]',  θ), 1:n)
        v = sum([ v[i] for i in 1:n  ])
        v = [v...]
        return v
    end
    obj = θ -> transpose(sample_moment(Y,X,Z,θ)) * W * sample_moment(Y,X,Z,θ)
    
    #Initial Value 
    theta0 = zeros(size(X,2))

    # We set the criterion function as an instance that we can differentiate twice
    td = TwiceDifferentiable(obj, theta0 ; autodiff = :forward)
    o = optimize(td, theta0, Newton(), Optim.Options() )

    if !Optim.converged(o)
        error("Minimization failed.")
    end

    θhat = Optim.minimizer(o)


    # Get asyvar, we compute the gradient of g with respect to theta
    # Jacobian will yield dg(W,θ)/dθ' (lxk matrix)
    v = map(i -> ForwardDiff.jacobian(θ -> g(Y[i],X[i, :]',Z[i,:]', θ), θhat), 1:n)
    dg = sum([ v[i] for i in 1:n ])
   
    v = map(i -> g(Y[i],X[i, :]',Z[i,:]', θhat), 1:n)
    outerprod = sum([ [v[i]...]*[v[i]...]' for i in 1:n ])
    
    mmd = dg' *W* dg
    avar = (mmd/n) \ dg' *W*(outerprod/n)*W*dg / (mmd/n)

    se = sqrt.(diag(avar))


    return (θhat=θhat , se = se )
end

estimate_nlgmm (generic function with 1 method)

In [16]:
@unpack Xendo, Xexo,Y,Z1 = data_generating_gmm(n=1000)
X=hcat(Xendo,Xexo)
Z=hcat(Xexo,Z1)
W = inv(Z'*Z);

In [17]:
@unpack θhat, se = estimate_nlgmm(Y, X, Z,W) 

(θhat = [1.0220958428305529, 0.9423644898707992, 0.9718141816923822, 0.9672702116728052], se = [0.8008974330527712, 1.458818492880134, 1.3239302309718795, 1.4669834816238212])

In [18]:
n=1000
R=10^3

θcoverage95 = zeros(R)

Threads.@threads for r=1:R
    
    @unpack Xendo, Xexo,Y,Z1 = data_generating_gmm(;n=n)
    X=hcat(Xendo,Xexo)
    Z=hcat(Xexo,Z1)
    @unpack θhat, se = estimate_nlgmm(Y, X, Z,W) 
    
    lower_ci  = θhat .- 1.96.*se/sqrt(n)
    upper_ci  = θhat .+ 1.96.*se/sqrt(n)
    θcoverage95[r] = (lower_ci[1]<1.0)*(upper_ci[1]>1.0)
end

sum(θcoverage95)/R

0.94

## 4. MLE

 - Population Criterion : $Q(\theta) = -\mathbf{E} \log f(W_i,\theta)$ 
 - Sample Criterion: $Q_n(\theta) = -\frac{1}{n}   \sum_{i=1}^n f(W_i,\theta) $
 - Gradient:  $\frac{\partial Q_n(\theta_0)}{\partial \theta} = -\frac{1}{n}   \sum_{i=1}^n \frac{\partial}{\partial \theta} f(W_i,\theta_0)$
 - Hessian: $\frac{\partial Q_n(\theta_0)}{\partial \theta \partial \theta'} = -\frac{1}{n}   \sum_{i=1}^n \frac{\partial^2}{\partial \theta \partial \theta'} f(W_i,\theta)$
 $\rightarrow_p -\mathbf{E} \frac{\partial^2}{\partial \theta \partial \theta'} \log f(W_i,\theta_0)  $ 
 - Influence function: $\xi_i := \left[ \mathbf{E} \frac{\partial^2}{\partial \theta \partial \theta'} \log f(W_i,\theta_0)  \right]^{-1} \frac{\partial}{\partial \theta} \log f(W_i,\theta_0)  $ 
 
Therefore, the asymptotic variance is given by 
$ \left[ \mathbf{E} \frac{\partial^2}{\partial \theta \partial \theta'} \log f(W_i,\theta_0)  \right]^{-1} \mathbf{E}[ \frac{\partial}{\partial \theta} \log f(W_i,\theta_0) \frac{\partial}{\partial \theta'} \log f(W_i,\theta_0) ] \left[ \mathbf{E} \frac{\partial^2}{\partial \theta \partial \theta'} \log f(W_i,\theta_0)  \right]^{-1} $

Under correctly specified MLE the matrix inside $\Omega_0$ equals the Hessian $B_0$, and the asymptotic variance is reduced to the inverse of the Hessian matrix. If that is not the case, the minimizer is a pseudo-true parameter and this is called the Quasi-MLE approach.  

In [19]:
using Parameters, LinearAlgebra, Distributions, Optim, ForwardDiff

In [20]:
function data_generating_mle(;n=1000)
      
    X=randn(n)
    u=randn(n)
    Y=(u .< 1.0 .+ X)
    
    return (Y=Y,X=X)
end

Φ(v)=cdf(Normal(0,1),v)
ϕ(v)=pdf(Normal(0,1),v)

ϕ (generic function with 1 method)

In [21]:
function LogL(θ,X,Y)
    n=length(Y)
    LL=0.0
    for i=1:n
        indx=θ[1]+X[i]*θ[2]
        LL -=Y[i]*log(Φ(indx))+(1-Y[i])*log(1-Φ(indx))
    end
    return LL/n
end

function estimate_MLE(X,Y)
    result=optimize(θ->LogL(θ,X,Y),[0.0;0.0];autodiff = :forward)
    θhat = Optim.minimizer(result)
    
    Var = inv(ForwardDiff.hessian(θ->LogL(θ,X,Y), θhat))
        
    return (θhat=θhat, se = sqrt.(diag(Var)) )
end

estimate_MLE (generic function with 1 method)

In [22]:
@unpack X,Y = data_generating_mle(;n=1000)
@unpack θhat, se = estimate_MLE(X,Y)

(θhat = [0.9983252683833257, 1.0123227876139826], se = [1.8809879670840206, 2.1565735311544567])

In [23]:
n=1000
R=10^3

θcoverage95 = zeros(R)

Threads.@threads for r=1:R
    
    @unpack X,Y = data_generating_mle(;n=1000)
    @unpack θhat, se = estimate_MLE(X,Y)

    lower_ci  = θhat .- 1.96.*se/sqrt(n)
    upper_ci  = θhat .+ 1.96.*se/sqrt(n)
    θcoverage95[r] = (lower_ci[1]<1.0)*(upper_ci[1]>1.0)
end

sum(θcoverage95)/R

0.945

## 5. Non-linear Least Squares

 - Population Criterion : $Q(\theta) = \mathbf{E} \left[ (Y_i - g(X_i,\theta))^2 \right]/2$
 - Sample Criterion: $Q_n(\theta) = \frac{1}{n} \sum_{i=1}^n (Y_i - g(X_i,\theta))^2/2$
 - Gradient:  $\frac{\partial Q_n(\theta_0)}{\partial \theta} = - \frac{1}{n} \sum_{i=1}^n \frac{\partial}{\partial \theta} g(X_i,\theta_0) (Y_i-g(X_i,\theta_0))$
 - Hessian: $\frac{\partial Q_n(\theta_0)}{\partial \theta \partial \theta'} = \frac{1}{n} \sum_{i=1}^n \{ \frac{\partial}{\partial \theta} g(X_i,\theta_0) \frac{\partial}{\partial \theta} g(X_i,\theta_0)' - U_i \frac{\partial^2}{\partial \theta \partial \theta'} g(X_i,\theta_0) \}  \rightarrow_p \mathbf{E}\left[ \frac{\partial}{\partial \theta} g(X_i,\theta_0) \frac{\partial}{\partial \theta} g(X_i,\theta_0)'\right] - \mathbf{E} \left[ U_i \frac{\partial^2}{\partial \theta \partial \theta'} g(X_i,\theta_0)\right]$
 
 where the second component is equal to 0 if the model is correctly specified $\mathbf{E}[U_i \mid X_i] =0$ and the law of iterated expectations. I will assume the model is correctly specified to simplify the algebra below. 
 
 - Influence function: $\xi_i :=   (\mathbf{E}\left[ \frac{\partial}{\partial \theta} g(X_i,\theta_0) \frac{\partial}{\partial \theta} g(X_i,\theta_0)'\right])^{-1} \frac{\partial}{\partial \theta} g(X_i,\theta_0) U_i$ 
 
Therefore, the asymptotic variance is given by 
$  (\mathbf{E}\left[ \frac{\partial}{\partial \theta} g(X_i,\theta_0) \frac{\partial}{\partial \theta} g(X_i,\theta_0)'\right])^{-1} \mathbf{E}[ U_i^2 \frac{\partial}{\partial \theta} g(X_i,\theta_0) \frac{\partial}{\partial \theta} g(X_i,\theta_0)']  (\mathbf{E}\left[ \frac{\partial}{\partial \theta} g(X_i,\theta_0) \frac{\partial}{\partial \theta} g(X_i,\theta_0)'\right])^{-1}  $

In [24]:
using Parameters, LinearAlgebra, Distributions, Optim, ForwardDiff

In [25]:
function data_generating_nls(;n=1000)
    #Define the Multivariate Normal Distribution instance
    mvnormal = MvNormal([1.0; 2.0], [1 0.7; 0.7 1]);
    
    #Define Logistic Link function 
    g = (x, b) -> (1 .+ exp.(-x * b)) .^ -1
    
    #Matrix X (n x 2 ) and ε (n x 1)
    X = rand(mvnormal,n)
    X = transpose(X)
    ε = randn(n, 1)
    
    #Vector y
    Y = g(X,[1.0, -1.0]) + ε
        
    #Return the data
    return (X = X , Y = Y)
end

data_generating_nls (generic function with 1 method)

In [26]:
function estimate_nls(f, y, x ) 
    # Obtain sample size
    n = size(x, 1)


    # Objective function
    obj = b -> sum((y - f(x, b)) .^ 2);
    
    #Initial Value 
    beta0 = zeros(size(x,2))

    # We set the criterion function as an instance that we can differentiate twice
    td = TwiceDifferentiable(obj, beta0 ; autodiff = :forward)
    o = optimize(td, beta0, Newton(), Optim.Options() )

    if !Optim.converged(o)
        error("Minimization failed.")
    end

    θhat = Optim.minimizer(o)


    # Get residuals
    r_hat = y - f(x, θhat)

    # Get asyvar, we compute the gradient of f with respect to b
    v = map(i -> ForwardDiff.gradient(θ -> f(x[i, :]', θ), θhat), 1:n)
    md = vcat(v'...)

    me = md .* r_hat; mmd = md' * md
    avar = (mmd/n) \ (me' * me/n) / (mmd/n)

    se = sqrt.(diag(avar))

    return (θhat = θhat, se = se)
end

estimate_nls (generic function with 1 method)

In [27]:
@unpack X,Y = data_generating_nls()
g = (x, b) -> (1 .+ exp.(-x * b)) .^ -1

@unpack θhat, se = estimate_nls(g,Y,X)

(θhat = [0.8338776974612384, -0.8611441619742458], se = [7.943177241033485, 5.815116273622467])

In [28]:
n=1000
R=10^3

θcoverage95 = zeros(R)
g = (x, b) -> (1 .+ exp.(-x * b)) .^ -1

Threads.@threads for r=1:R
    
    @unpack X,Y = data_generating_nls(;n=n)
    @unpack θhat, se = estimate_nls(g,Y,X)
    
    lower_ci  = θhat .- 1.96.*se/sqrt(n)
    upper_ci  = θhat .+ 1.96.*se/sqrt(n)
    θcoverage95[r] = (lower_ci[1]<1.0)*(upper_ci[1]>1.0)
end

sum(θcoverage95)/R

0.949

## 6. Minimum Distance

 - Population Criterion : $Q(\theta) = (\pi_0 - g(\theta))' A'A (\pi_0 - g(\theta))/2$, 
 - Sample Criterion: $Q_n(\theta) = (\hat{\pi}_n - g(\theta))' A_n'A_n (\hat{\pi}_n - g(\theta))/2$
 
  where $\sqrt n (\hat{\pi}_n - \pi_0 ) \rightarrow_p \mathcal{N}(0,V_0)$ is a first-step estimator.

 - Gradient:  $\frac{\partial Q_n(\theta_0)}{\partial \theta} = - \left( \frac{\partial g(\theta_0)}{\partial \theta} \right)' A_n'A_n (\hat{\pi}_n - g(\theta_0))$
  - Hessian: $\frac{\partial Q_n(\theta_0)}{\partial \theta \partial \theta'} = \left( \frac{\partial g(\theta_0)}{\partial \theta} \right)' A_n'A_n  \left( \frac{\partial g(\theta_0)}{\partial \theta} \right) + \left[I_k \otimes (\hat{\pi}_n-g(\theta_0))' A_n'A_n \right] \frac{\partial}{\partial \theta'} vec\left(  \frac{\partial g(\theta_0)}{\partial \theta'}\right) $
$\rightarrow_p \Gamma_0 A'A\Gamma_0 + \left[ I_k \otimes (\pi_0-g(\theta_0))' A'A  \right] \frac{\partial}{\partial \theta'} vec\left(  \frac{\partial g(\theta_0)}{\partial \theta'}\right) $
 
 where the second component is equal to 0 if the model is correctly specified $\pi_0 = g(\theta_0)$. I will assume the model is correctly specified to simplify the algebra below. 
 
 - Influence function: $\xi_i :=   (\mathbf{E}\left[ \frac{\partial}{\partial \theta} g(X_i,\theta_0) \frac{\partial}{\partial \theta} g(X_i,\theta_0)'\right])^{-1} \frac{\partial}{\partial \theta} g(X_i,\theta_0) U_i$ 
 
Therefore, the asymptotic variance is given by 
$  ( \Gamma_0 A'A \Gamma_0 )^{-1} \Gamma_0 A'A \xi_i^{\pi} $, 
where $\xi_i^{\pi}$ is the influence function that comes from $\sqrt n (\hat{\pi}_n - \pi_0 )$ (i.e. the first step).


## 7. Quantile Regression 

Recall that quantile regression is a special case where we encounter differentiability issues. The approach taken here is to construct a stochastic equicontinuous process such that we obtain a smooth function where we can apply a mean value expansion around the true value $\theta_{\tau,0}$.

This approach yields the following asymptotic linear representation

\begin{align*}
\sqrt n (\hat{\theta}_{\tau,n} - \theta_{\tau,0} ) &= - \left( \mathbf{E}[f(X_i'\theta_{\tau,0}) X_i X_i']\right)^{-1} \frac{1}{\sqrt n} \sum_{i=1}^n \{  (\tau - \mathbf{1}\{ Y_i < X_i'\theta_{\tau,0} \} X_i \} + o_P(1)
\end{align*}

This implies that the influence function is given by 
$\xi_i = \left( \mathbf{E}[f(X_i'\theta_{\tau,0}) X_i X_i']\right)^{-1} \{  (\tau - \mathbf{1}\{ Y_i < X_i'\theta_{\tau,0} \} X_i \}$, and the asymptotic variance is given by $\left( \mathbf{E}[f(X_i'\theta_{\tau,0}) X_i X_i']\right)^{-1}\mathbf{E}[\tau (1-\tau) X_i X_i'] \left(    \mathbf{E}[f(X_i'\theta_{\tau,0}) X_i X_i']\right)^{-1}$ by the L.I.E.



In [29]:
using LinearAlgebra, JuMP, HiGHS, Optim, Statistics,Plots, Parameters

In [30]:
function data_generating_qr(;n=1000)
    X=randn(n,1).^2
    U=rand(n,1)
    Y=X .* (U .-0.5)
    return (Y=Y, X=X)
end

#If you wanna estimate asy-var
K(v)= abs(v) - 1 < 0 ?   0.5* (abs(v) - 1) : 0.0;

In [31]:
function estimate_qreg(;Y,X,τ)
    
    QR=Model(HiGHS.Optimizer)
    set_silent(QR)
    n=length(Y)
    
    c=[τ*ones(n,1);(1-τ)*ones(n,1);];
    @variable(QR,x[1:2*n]>=0.0); #this includes the non-negativity constraint on z^{+} and z^{-}
    @variable(QR,-100.0<=b<=100.0); #if you want to estimate vector b you use @variable(QR,-100.0<=b[1:k]<=100.0) where k is dimension
    @objective(QR,Min,sum(c[i]*x[i] for i in 1:(2*n)));
    @constraint(QR,constraint[i in 1:n],x[i]-x[n+i]+sum(X[i,:].*b)==Y[i]); #the equality constraint
    optimize!(QR);
    estr=value.(b)
    
    
    #Drop this if you don't care about asy-var
    "Calculation of Asymptotic Variance"
    n=length(Y)
    rhat= Y - X * estr;
    h = 1.06 * std(rhat) * n ^ (-1 / 5);
    Khat =  K.(rhat./h) ;

    Ωhat = τ * (1 - τ) * (X' * X);
    Bhat = ((X.*repeat(Khat,1,size(X,2)))' * X) ./ h;
    v = n*(Bhat \ Ωhat / Bhat);
    
    
    return (θhat = estr , se = sqrt.(diag(v)) )
end


estimate_qreg (generic function with 1 method)

In [32]:
@unpack Y, X =data_generating_qr();
@unpack θhat, se=estimate_qreg(;Y=Y,X=X,τ=0.25)

(θhat = -0.24898270284504034, se = [0.8658257109940635])

In [33]:
n=1000
R=10^3
τcheck=0.75
θτ=τcheck-0.5
θcoverage95 = zeros(R)

Threads.@threads for r=1:R
    
    @unpack Y, X =data_generating_qr(;n=n);
    @unpack θhat, se=estimate_qreg(;Y=Y,X=X,τ=τcheck)
    
    lower_ci  = θhat .- 1.96.*se/sqrt(n)
    upper_ci  = θhat .+ 1.96.*se/sqrt(n)
    θcoverage95[r] = (lower_ci[1]<θτ)*(upper_ci[1]>θτ)
end

sum(θcoverage95)/R

0.994

## Using the ECON627 package we can use some of these functions that are precompiled

In [35]:
using ECON627_UBC
# You don't need to hard code estimators anymore :) 
@unpack X,y = data_generating_ols(n=1000)
@unpack b, se = ols(X,y)

(b = [1.0663071246108842; 0.9651268464189884;;], se = [0.0013242315493568438, 0.0008359902945974531])